<a href="https://colab.research.google.com/github/itspallavi20/Oral_Diseases_Prediction/blob/main/models/MobileNet_V2_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/Oral_Diseases_Project/Dataset/Train'
val_dir = '/content/drive/MyDrive/Oral_Diseases_Project/Dataset/Validation'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

img_size = 224

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 1479 images belonging to 6 classes.
Found 300 images belonging to 6 classes.


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

base_model.trainable = True
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

 #The model after training
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 132s 3s/step - accuracy: 0.9532 - loss: 0.1472 - val_accuracy: 0.0400 - val_loss: 6.6024
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 129s 3s/step - accuracy: 0.9656 - loss: 0.1331 - val_accuracy: 0.0400 - val_loss: 6.7339
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 125ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.9699 - loss: 0.1275 - val_accuracy: 0.0367 - val_loss: 6.8317
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 130s 3s/step - accuracy: 0.9787 - loss: 0.0929 - val_accuracy: 0.0367 - val_loss: 6.9133
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.9757 - loss: 0.1000 - val_accuracy: 0.0300 - val_loss

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

confusion_mat = np.array([
    [0, 12, 6, 4, 0, 1],
    [0, 11, 6, 1, 0, 0],
    [0, 34, 10, 10, 1, 2],
    [0, 84, 42, 24, 1, 2],
    [0, 14, 9, 6, 1, 0],
    [0, 12, 3, 4, 0, 0]
])

num_classes = confusion_mat.shape[0]

metrics = {class_label: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for class_label in range(num_classes)}

for i in range(num_classes):
    metrics[i]["TP"] = confusion_mat[i, i]  # Diagonal element is TP
    metrics[i]["FP"] = np.sum(confusion_mat[:, i]) - confusion_mat[i, i]  # Column sum - TP
    metrics[i]["FN"] = np.sum(confusion_mat[i, :]) - confusion_mat[i, i]  # Row sum - TP
    metrics[i]["TN"] = np.sum(confusion_mat) - (metrics[i]["TP"] + metrics[i]["FP"] + metrics[i]["FN"])  # Total sum - (TP + FP + FN)

print("Confusion Matrix:")
print(confusion_mat)

print("\nMetrics for Each Class:")
for class_label in range(num_classes):
    print(f"Class {class_label}:")
    print(f"  TP: {metrics[class_label]['TP']}")
    print(f"  FP: {metrics[class_label]['FP']}")
    print(f"  FN: {metrics[class_label]['FN']}")
    print(f"  TN: {metrics[class_label]['TN']}")

print("\nSensitivity (Recall) and Specificity for Each Class:")
for class_label in range(num_classes):
    TP = metrics[class_label]["TP"]
    FP = metrics[class_label]["FP"]
    FN = metrics[class_label]["FN"]
    TN = metrics[class_label]["TN"]

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0  # Avoid division by zero
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0  # Avoid division by zero

    print(f"Class {class_label}:")
    print(f"  Sensitivity (Recall): {sensitivity:.4f}")
    print(f"  Specificity: {specificity:.4f}")


Confusion Matrix:
[[ 0 12  6  4  0  1]
 [ 0 11  6  1  0  0]
 [ 0 34 10 10  1  2]
 [ 0 84 42 24  1  2]
 [ 0 14  9  6  1  0]
 [ 0 12  3  4  0  0]]

Metrics for Each Class:
Class 0:
  TP: 0
  FP: 0
  FN: 23
  TN: 277
Class 1:
  TP: 11
  FP: 156
  FN: 7
  TN: 126
Class 2:
  TP: 10
  FP: 66
  FN: 47
  TN: 177
Class 3:
  TP: 24
  FP: 25
  FN: 129
  TN: 122
Class 4:
  TP: 1
  FP: 2
  FN: 29
  TN: 268
Class 5:
  TP: 0
  FP: 5
  FN: 19
  TN: 276

Sensitivity (Recall) and Specificity for Each Class:
Class 0:
  Sensitivity (Recall): 0.0000
  Specificity: 1.0000
Class 1:
  Sensitivity (Recall): 0.6111
  Specificity: 0.4468
Class 2:
  Sensitivity (Recall): 0.1754
  Specificity: 0.7284
Class 3:
  Sensitivity (Recall): 0.1569
  Specificity: 0.8299
Class 4:
  Sensitivity (Recall): 0.0333
  Specificity: 0.9926
Class 5:
  Sensitivity (Recall): 0.0000
  Specificity: 0.9822
